In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%%shell
pip install datasets -q
pip install pyserini -q
pip install faiss-cpu -q
pip install evaluate -q
pip install trectools -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00

In [3]:
%%shell
apt-get install maven -qq
git clone --recurse-submodules https://github.com/castorini/pyserini.git
cd pyserini
cd tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
cd tools/eval/ndeval && make && cd ../../..

Extracting templates from packages: 100%
Selecting previously unselected package libapache-pom-java.
(Reading database ... 123069 files and directories currently installed.)
Preparing to unpack .../00-libapache-pom-java_18-1_all.deb ...
Unpacking libapache-pom-java (18-1) ...
Selecting previously unselected package libatinject-jsr330-api-java.
Preparing to unpack .../01-libatinject-jsr330-api-java_1.0+ds1-5_all.deb ...
Unpacking libatinject-jsr330-api-java (1.0+ds1-5) ...
Selecting previously unselected package libgeronimo-interceptor-3.0-spec-java.
Preparing to unpack .../02-libgeronimo-interceptor-3.0-spec-java_1.0.1-4fakesync_all.deb ...
Unpacking libgeronimo-interceptor-3.0-spec-java (1.0.1-4fakesync) ...
Selecting previously unselected package libcdi-api-java.
Preparing to unpack .../03-libcdi-api-java_1.2-2_all.deb ...
Unpacking libcdi-api-java (1.2-2) ...
Selecting previously unselected package libcommons-cli-java.
Preparing to unpack .../04-libcommons-cli-java_1.4-1_all.deb ...

In [4]:
import os
import json
import numpy as np
import pandas as pd
import evaluate
import collections

from tqdm import tqdm
from time import time
from datasets import load_dataset
from collections import defaultdict, Counter
from pyserini.search.lucene import LuceneSearcher

In [5]:
# select folderpath
user = "leonardo" # monique
if user == "monique":
    main_dir = '/content/gdrive/MyDrive/Unicamp-projeto-final/'
else:
    main_dir = '/content/gdrive/MyDrive/Unicamp/IA368-DD/'

In [6]:
# download data
collection = load_dataset('unicamp-dl/mrobust', 'collection-portuguese')
queries = load_dataset('unicamp-dl/mrobust', 'queries-portuguese')
!wget https://huggingface.co/datasets/unicamp-dl/mrobust/raw/main/qrels.robust04.txt

Generating collection split:   0%|          | 0/528032 [00:00<?, ? examples/s]

Dataset mrobust downloaded and prepared to /root/.cache/huggingface/datasets/unicamp-dl___mrobust/collection-portuguese/1.0.0/a91c748e4ba08987678ac4eebaf08238bf6ee876e4ac74d4b996cbbc7b10014d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating queries split:   0%|          | 0/250 [00:00<?, ? examples/s]

Dataset mrobust downloaded and prepared to /root/.cache/huggingface/datasets/unicamp-dl___mrobust/queries-portuguese/1.0.0/a91c748e4ba08987678ac4eebaf08238bf6ee876e4ac74d4b996cbbc7b10014d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

--2023-06-28 23:43:30--  https://huggingface.co/datasets/unicamp-dl/mrobust/raw/main/qrels.robust04.txt
Resolving huggingface.co (huggingface.co)... 13.35.166.114, 13.35.166.69, 13.35.166.36, ...
Connecting to huggingface.co (huggingface.co)|13.35.166.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6543541 (6.2M) [text/plain]
Saving to: ‘qrels.robust04.txt’

qrels.robust04.txt  100%[===================>]   6.24M  5.17MB/s    in 1.2s    

2023-06-28 23:43:32 (5.17 MB/s) - ‘qrels.robust04.txt’ saved [6543541/6543541]



In [7]:
# build corpus
output_tsv = f'/content/corpus.tsv'
with open(output_tsv,'w') as f_out:
    for line in tqdm(collection["collection"], desc=f'Writing file on {output_tsv}'):
        f_out.write(f'{line["id"]}\t{line["text"]}\n')

Writing file on /content/corpus.tsv: 100%|██████████| 528032/528032 [00:44<00:00, 11829.56it/s]


In [8]:
!wc -l corpus.tsv

528032 corpus.tsv


In [9]:
!head -n 10 corpus.tsv

FT931-16655	930105 FT 05 JAN 93 / Redland concorda acordos europeus de 59 milhões de libras REDLAND, o grupo de materiais de construção que pagou 624 milhões de libras em ações pela Steetley no ano passado, arrecadou 100 milhões de libras por meio de alienações. Está reinvestindo 58,6 milhões de libras em suas atividades de azulejos e tijolos na Europa continental. A Redland está formando uma joint venture com a Koramic, uma empresa privada belga, que engloba as fábricas de tijolos das duas empresas na Holanda, Alemanha e Bélgica. A nova empresa será a maior fabricante de tijolos de fachada da Europa continental. Os lucros das atividades combinadas em 1992 teriam sido de 15,9 milhões de libras, dos quais a parte de Redland teria sido de 5,8 milhões de libras. A joint venture 50:50 deterá 70 por cento das atividades combinadas, dando a Redland uma participação econômica de 35 por cento, mas controle conjunto. Redland também receberá 17,3 milhões de libras em dinheiro. A Redland está aum

In [10]:
df = pd.read_csv("/content/corpus.tsv", sep = "\t", header = None)
df.columns = ["_id", "text"]
corpus = df[["_id", "text"]].set_index("_id").to_dict(orient = 'dict')["text"]

In [11]:
# build queries
output_tsv = f'/content/queries.tsv'
with open(output_tsv,'w') as f_out:
    for line in tqdm(queries["queries"], desc=f'Writing file on {output_tsv}'):
        f_out.write(f'{line["id"]}\t{line["text"]}\n')

Writing file on /content/queries.tsv: 100%|██████████| 250/250 [00:00<00:00, 18478.74it/s]


In [12]:
topics = pd.read_csv("/content/queries.tsv", sep = "\t", header = None)
topics.columns = ["_id", "text"]
topics["_id"] = topics["_id"].astype(str)
topics = topics[["_id", "text"]].set_index("_id").to_dict(orient = 'dict')["text"]

In [13]:
# build qrels
output_tsv = f'/content/qrels.tsv'
with open('/content/qrels.robust04.txt', 'r') as input_file:
    with open(output_tsv, 'w', newline='') as output_file:
        for line in input_file:
            #output_file.write(line)
            output_file.write(line.replace(" ", "\t"))

In [14]:
!wc -l /content/qrels.robust04.txt

311410 /content/qrels.robust04.txt


In [15]:
#qrels = pd.read_csv("/content/qrels.tsv", sep = "\t", header = None)
qrels = pd.read_csv("/content/qrels.tsv", sep = "\t", header = None)
qrels.columns = ["query_id", "0", "doc_id", "rel"]
qrels["query_id"] = qrels["query_id"].astype(str)

In [16]:
!head -n 10 /content/qrels.tsv

301	0	FBIS3-10082	1
301	0	FBIS3-10169	0
301	0	FBIS3-10243	1
301	0	FBIS3-10319	0
301	0	FBIS3-10397	1
301	0	FBIS3-10491	1
301	0	FBIS3-10555	0
301	0	FBIS3-10622	1
301	0	FBIS3-10634	0
301	0	FBIS3-10635	0


### BM25 Indexing

In [17]:
%%shell
rm -rf /content/collections
mkdir /content/collections

In [18]:
!rm -rf /content/indexes/lucene-index-trec-mrobust

In [19]:
# Save corpus as jsonl file
output_json = f'/content/collections/corpus.jsonl'

with open(output_json,'w') as f_out:
    for line in tqdm(collection["collection"], desc=f'Writing file on {output_tsv}'):
        output_dict = {'id': line["id"], 'contents': line["text"]}
        f_out.write(json.dumps(output_dict, ensure_ascii=False) + '\n')

Writing file on /content/qrels.tsv: 100%|██████████| 528032/528032 [01:16<00:00, 6857.89it/s]


In [20]:
collection["collection"]['id'][-1]

'LA021690-0192'

In [21]:
collection["collection"]['text'][-1]

'<P> 16 de fevereiro de 1990, sexta-feira, edição doméstica </P> <P> SEM CERIMÔNIA, ELLIOTT ESTÁ SUPERANDO; </P> <P> PISTA E CAMPO: ELE É O HERDADEIRO APARENTE PARA A TRADIÇÃO BRITÂNICA DE MÉDIA DISTÂNCIA. </P> <P> Uma nova década é o sinal para uma mudança da guarda nas estimadas fileiras dos corredores de meia distância britânicos. </P> <P> A década de 1980 pertenceu a Sebastian Coe, Steve Ovett e Steve Cram, cujas realizações fazem parte da tradição do atletismo. </P> <P> Nos anos 90, Cram, o detentor do recorde mundial nesta milha, provavelmente ainda será um fator, junto com um carpinteiro siderúrgico de 27 anos que está emergindo como talvez o próximo grande Corredor britânico. </P> <P> Peter Elliott, que correrá a milha esta noite no Times / Eagle Indoor Games no Fórum, serviu seu aprendizado e está aparentemente pronto para ser aclamado internacionalmente. </P> <P> A temporada de atletismo apenas começou, mas Elliott já venceu os 1.500 metros nos Jogos da Commonwealth em Auckla

In [22]:
!wc -l /content/corpus.tsv

528032 /content/corpus.tsv


In [23]:
!wc -l /content/collections/corpus.jsonl

528032 /content/collections/corpus.jsonl


In [24]:
!head -n 100 /content/collections/corpus.jsonl

{"id": "FT931-16655", "contents": "930105 FT 05 JAN 93 / Redland concorda acordos europeus de 59 milhões de libras REDLAND, o grupo de materiais de construção que pagou 624 milhões de libras em ações pela Steetley no ano passado, arrecadou 100 milhões de libras por meio de alienações. Está reinvestindo 58,6 milhões de libras em suas atividades de azulejos e tijolos na Europa continental. A Redland está formando uma joint venture com a Koramic, uma empresa privada belga, que engloba as fábricas de tijolos das duas empresas na Holanda, Alemanha e Bélgica. A nova empresa será a maior fabricante de tijolos de fachada da Europa continental. Os lucros das atividades combinadas em 1992 teriam sido de 15,9 milhões de libras, dos quais a parte de Redland teria sido de 5,8 milhões de libras. A joint venture 50:50 deterá 70 por cento das atividades combinadas, dando a Redland uma participação econômica de 35 por cento, mas controle conjunto. Redland também receberá 17,3 milhões de libras em dinhe

In [25]:
!tail -n 100 /content/collections/corpus.jsonl

{"id": "LA021690-0093", "contents": "<P> 16 de fevereiro de 1990, sexta-feira, Valley Edition </P> <P> CAMPUS CELEBRATES BLACK ARTS; </P> <P> HISTÓRIA: UMA CELEBRAÇÃO MENSAL TERÁ DANÇA, DRAMA, CANÇÃO NO CAL STATE NORTHRIDGE CAMPUS. </P> <P> As artes e cultura negras serão celebradas em uma série de eventos planejados pelo Departamento de Estudos Pan-africanos em Cal State Northridge e várias organizações estudantis para comemorar o Mês dos Estudos Negros. </P> <P> Os eventos incluem um programa chamado \"The Black Star Search\", um programa patrocinado das 7h às 9h30 pela Delta Sigma Theta, uma irmandade negra da CSUN. </P> <P> O coordenador Lashawn Kindle disse que o evento, inspirado no popular programa de televisão \"Star Search\", incluirá oito áreas de competição: porta-voz e porta-voz, vocalista masculino e feminino, dançarino e dançarina, leitura dramática e rap intérpretes de música. </P> <P> Como esta é a segunda competição anual desse tipo, os campeões do ano passado competir

In [26]:
# Faz a indexação dos docs gerados acima como uma JsonCollection utilizando o Pyserini
!python -m pyserini.index.lucene --collection JsonCollection --input /content/collections --language pt --index indexes/lucene-index-trec-mrobust --generator DefaultLuceneDocumentGenerator --threads 9 --storePositions --storeDocvectors --storeRaw

2023-06-28 23:46:49,489 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-06-28 23:46:49,493 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-06-28 23:46:49,493 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-06-28 23:46:49,493 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/collections
2023-06-28 23:46:49,494 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-06-28 23:46:49,494 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-06-28 23:46:49,495 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-06-28 23:46:49,495 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: pt
2023-06-28 23:46:49,496 INFO  [main] index.IndexCollection (I

In [27]:
# Utiliza o pacote pyserini para ranquear o msmarco-passage.txt
!python -m pyserini.search.lucene \
    --topics "/content/queries.tsv" \
    --index "/content/indexes/lucene-index-trec-mrobust" \
    --language pt \
    --output run.mrobust.txt \
    --output-format trec \
    --hits 1000 \
    --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running /content/queries.tsv topics, saving to run.mrobust.txt...
100% 250/250 [01:11<00:00,  3.52it/s]


In [40]:
!/content/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -m recip_rank -m map -m ndcg_cut -m recall.1000 /content/qrels.tsv /content/run.mrobust.txt

map                   	all	0.2307
recip_rank            	all	0.6690
recall_1000           	all	0.6572
ndcg_cut_5            	all	0.4388
ndcg_cut_10           	all	0.4098
ndcg_cut_15           	all	0.3967
ndcg_cut_20           	all	0.3893
ndcg_cut_30           	all	0.3798
ndcg_cut_100          	all	0.3909
ndcg_cut_200          	all	0.4201
ndcg_cut_500          	all	0.4645
ndcg_cut_1000         	all	0.4940


In [29]:
!wc -l run.mrobust.txt

250000 run.mrobust.txt


In [30]:
!head -n 10 run.mrobust.txt

301 Q0 FBIS3-39566 1 16.766001 Anserini
301 Q0 FBIS3-41385 2 16.765999 Anserini
301 Q0 FBIS4-56982 3 15.860300 Anserini
301 Q0 FBIS4-67072 4 15.860299 Anserini
301 Q0 FBIS3-41285 5 15.858900 Anserini
301 Q0 FBIS3-42315 6 15.858899 Anserini
301 Q0 FBIS4-7811 7 15.637700 Anserini
301 Q0 FBIS4-42757 8 15.572900 Anserini
301 Q0 FBIS4-45419 9 15.572899 Anserini
301 Q0 FBIS4-46425 10 15.572898 Anserini


### Outra abordagem

In [ ]:
bm25_run = pd.read_csv("/content/run.mrobust.txt", sep = " ", header = None)
bm25_run.columns = ["query_id", "Q0", "doc_id", "rank", "score", "system"]

In [ ]:
def hits_to_trec_format(trec_eval, qrels_path, hits):

    qrels_format = defaultdict(list)

    with open(qrels_path, "r") as f:
        for line in f:
            qid, _, doc_id, relevance = line.strip().split("\t")
            qrels_format['query'].append(qid)
            qrels_format['q0'].append("q0")
            qrels_format['docid'].append(doc_id)
            qrels_format['rel'].append(relevance)

    run_format = defaultdict(list)

    for row in hits.iterrows():
      query_id, q_0, doc_id, rank, score, system = row[1]
      run_format['query'].append(query_id)
      run_format['q0'].append(q_0)
      run_format['docid'].append(doc_id)
      run_format['rank'].append(rank)
      run_format['score'].append(score)
      run_format['system'].append(system)

    trec_eval = evaluate.load("trec_eval")
    results = trec_eval.compute(predictions=[run_format], references=[qrels_format])
    return results

In [ ]:
# init
qrels_path = "/content/qrels.tsv"
trec_eval = evaluate.load("trec_eval")
bm25_eval = hits_to_trec_format(trec_eval, qrels_path, bm25_run[bm25_run["query_id"]!= 672].copy())
bm25_eval

In [34]:
def bm25_retrieval(bm25_seacher, query, k = 1000):

    # settings
    hits = collections.OrderedDict()

    # bm25 retrieval
    for query_id, query_text in query.items():
        docs_rel = {f.docid: f.score for f in bm25_seacher.search(query_text, k)}
        hits[query_id] = collections.OrderedDict(docs_rel)

    return hits

In [35]:
# init
qrels_path = "/content/qrels.tsv"
trec_eval = evaluate.load("trec_eval")

# prepare dicts
bm25_run = {}

# bm25 settings
bm25_indexes_path = "/content/indexes/lucene-index-trec-mrobust"
bm25_seacher = LuceneSearcher(bm25_indexes_path)
bm25_seacher.set_bm25(k1=0.82, b=0.68)
bm25_seacher.set_language("pt")

for query_id, query_text in tqdm(topics.items(), desc="BM25 Retrieval"):
    query = {query_id: query_text}
    query_hits_bm25 = bm25_retrieval(bm25_seacher, query, k = 1000)
    bm25_run[query_id] = query_hits_bm25.get(query_id)

BM25 Retrieval: 100%|██████████| 250/250 [00:55<00:00,  4.51it/s]


In [36]:
def hits_to_trec_format_python(trec_eval, qrels_path, hits, system):

    qrels_format = defaultdict(list)

    with open(qrels_path, "r") as f:
        for line in f:
            qid, _, doc_id, relevance = line.strip().split("\t")
            qrels_format['query'].append(qid)
            qrels_format['q0'].append("q0")
            qrels_format['docid'].append(doc_id)
            qrels_format['rel'].append(relevance)

    run_format = defaultdict(list)

    for query_id in hits:
      if query_id != '672':
        rank = 1
        for doc_id in hits.get(query_id):
            run_format['query'].append(query_id)
            run_format['q0'].append("q0")
            run_format['docid'].append(doc_id)
            run_format['rank'].append(rank)
            run_format['score'].append(hits.get(query_id).get(doc_id))
            run_format['system'].append(system)
            rank += 1

    trec_eval = evaluate.load("trec_eval")
    results = trec_eval.compute(predictions=[run_format], references=[qrels_format])
    return results

In [37]:
# init
qrels_path = "/content/qrels.tsv"
trec_eval = evaluate.load("trec_eval")
bm25_eval = hits_to_trec_format_python(trec_eval, qrels_path, bm25_run, "BM25")
bm25_eval

{'runid': 'BM25',
 'num_ret': 249000,
 'num_rel': 17412,
 'num_rel_ret': 8869,
 'num_q': 249,
 'map': 0.23068659041255937,
 'gm_map': 0.11676114361055182,
 'bpref': 0.25357563321146276,
 'Rprec': 0.26560985533506504,
 'recip_rank': 0.6690365552026022,
 'P@5': 0.4506024096385543,
 'P@10': 0.39036144578313253,
 'P@15': 0.3526104417670683,
 'P@20': 0.3271084337349397,
 'P@30': 0.28313253012048195,
 'P@100': 0.16401606425702814,
 'P@200': 0.10977911646586345,
 'P@500': 0.05945381526104418,
 'P@1000': 0.03561847389558233,
 'NDCG@5': 0.43880662531878223,
 'NDCG@10': 0.409834345734068,
 'NDCG@15': 0.39667860422215995,
 'NDCG@20': 0.3892548255469207,
 'NDCG@30': 0.3797916691912629,
 'NDCG@100': 0.3909150040477435,
 'NDCG@200': 0.42012862812420393,
 'NDCG@500': 0.4644951677315255,
 'NDCG@1000': 0.49400906545427653}